In [ ]:
import os

os.environ["KERAS_BACKEND"] = "torch"

import keras

import lightning as L
import torch
from torch import nn
from torch.utils.data import DataLoader

from hympi_ml.data import DataSpec, AMPRSpec, NRSpec
from hympi_ml.data.ch06 import Ch06Source
from hympi_ml.data.model_dataset import ModelDataset


class SpecModel(L.LightningModule):
    def __init__(
        self,
        features: dict[str, DataSpec],
        targets: dict[str, DataSpec],
        feature_paths: dict[str, nn.Module],
        output_path: nn.Module,
        loss,
    ):
        super().__init__()
        self.features = features
        self.targets = targets
        self.feature_paths = feature_paths
        self.output_path = output_path
        self.loss = loss

    def forward(self, inputs):
        features_forward = [path(inputs[k]) for k, path in self.feature_paths.items()]
        return self.output_path(torch.concat(features_forward))

    def training_step(self, batch, batch_idx):
        inputs, target = batch
        output = self(inputs)
        return nn.functional.mse_loss(output, target["TEMPERATURE"])

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

features={"AMPR": AMPRSpec()}
targets={"TEMPERATURE": NRSpec(dataset="TEMPERATURE")}

model = SpecModel(
    features=features,
    targets=targets,
    feature_paths={
        "AMPR": nn.Sequential(
            nn.Linear(8, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 32),
        )
    },
    output_path=nn.Sequential(
        nn.Linear(32, 72),
    ),
    loss=nn.MSELoss(),
)

dataset = ModelDataset(Ch06Source(days=["20060115"]), features=features, targets=targets, batch_size=16384)

loader = DataLoader(dataset, batch_size=None, shuffle=True, num_workers=10)

trainer = L.Trainer(enable_progress_bar=True, max_epochs=1, enable_model_summary=True)
trainer.fit(model=model, train_dataloaders=loader)

/explore/nobackup/people/dgershm1/hympi-ml-retrieval/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name        | Type       | Params | Mode 
---------------------------------------------------
0 | output_path | Sequential | 2.4 K  | train
1 | loss        | MSELoss    | 0      | train
---------------------------------------------------
2.4 K     Trainable params
0         Non-trainable params
2.4 K     Total params
0.010     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 1182/1182 [01:45<00:00, 11.17it/s, v_num=2]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1182/1182 [01:46<00:00, 11.06it/s, v_num=2]


In [ ]:
model